In [20]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf
tf.enable_eager_execution()

import librosa
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import librosa
import simpleaudio as sa
import math
from IPython import display
import sys

print("Finished imports")

Finished imports


In [17]:
for i in range(0, 100000):
    print(i, end='\r')

4867

11405

23902

25750

27372

32857

36903

KeyboardInterrupt: 

In [19]:
# Converting into spectrograms
# Spectrograms are stored as spectrograms.tfrecord
nsynth = tf.data.TFRecordDataset("nsynth-train.tfrecord")
i = 1
numTrue = 0
read_features = {
    'note': tf.FixedLenFeature([], dtype=tf.int64),
    'note_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_str': tf.FixedLenFeature([], dtype=tf.string),
    'pitch': tf.FixedLenFeature([], dtype=tf.int64),
    'velocity': tf.FixedLenFeature([], dtype=tf.int64),
    'sample_rate': tf.FixedLenFeature([], dtype=tf.int64),
    'audio': tf.VarLenFeature(dtype=float),
    'instrument_family': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_family_str': tf.FixedLenFeature([], dtype=tf.string),
    'instrument_source': tf.FixedLenFeature([], dtype=tf.int64),
    'instrument_source_str': tf.FixedLenFeature([], dtype=tf.string)
}
with tf.python_io.TFRecordWriter('spectrograms.tfrecord') as writer:
    acoustic = 0
    electronic = 0
    synthesized = 0
    not_sr = 0
    total = 0
    true_max = 0
    true_min = 100000
    phase_max = -100
    phase_min = 100
    for b in nsynth:
        data = tf.parse_single_example(serialized=b, features=read_features)
        #if(data['sample_rate'].numpy()!=16000):
        #    not_sr += 1
        #elif(data['instrument_source'].numpy()==0):
        #    acoustic += 1
        #elif(data['instrument_source'].numpy()==1):
        #    electronic += 1
        #elif(data['instrument_source'].numpy()==2):
        #    synthesized += 1
        total += 1
        #print("Invalid: {}, acoustic: {}, electronic: {}, synthesized: {}, total: {}, minimum: {}, maximum: {}, sample rate: {}".format(not_sr, acoustic, electronic, synthesized, total, true_min, true_max, data['sample_rate']), end='\r')
        if(total%1000==0): 
            print("Total: {}".format(total), end='\r')
        #if(True):
        if(data['instrument_source'].numpy()==0):
            audio = data['audio'].values.numpy()*15000
            audio = np.array(audio)
            spectrogram = librosa.stft(audio, 2048)
            
            magnitudes = np.log(np.abs(spectrogram) + 1.2664166e-14).reshape(1025, 126, 1)
            phases = np.angle(spectrogram).reshape(1025, 126, 1)
            #if(np.amax(magnitudes)>true_max):
            #    true_max = np.amax(magnitudes)
            #minimum = np.amin(magnitudes)
            #if(minimum < true_min):
            #    true_min = minimum
            magnitudes = 2*(magnitudes+32)/(16+32)-1
            #magnitudes = 2*(magnitudes-minimum)/(np.amax(magnitudes)-minimum)-1
            #if(np.amax(phases)>phase_max):
            #    phase_max = np.amax(phases)
            #if(np.amin(phases)<phase_min):
            #    phase_min = np.amin(phases)
            phases = phases / (math.pi)
            spec = np.concatenate([magnitudes, phases], 2).flatten()

            example = tf.train.Example(features=tf.train.Features(feature={
                'note': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['note'].numpy()])),
                'note_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['note_str'].numpy()])),
                'instrument': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument'].numpy()])),
                'instrument_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_str'].numpy()])),
                'pitch': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['pitch'].numpy()])),
                'velocity': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['velocity'].numpy()])),
                'sample_rate': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['sample_rate'].numpy()])),
                'spectrogram': tf.train.Feature(float_list=tf.train.FloatList(value=spec)),
                'instrument_family': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_family'].numpy()])),
                'instrument_family_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_family_str'].numpy()])),
                'instrument_source': tf.train.Feature(int64_list=tf.train.Int64List(value=[data['instrument_source'].numpy()])),
                'instrument_source_str': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data['instrument_source_str'].numpy()]))
            }))
            
            writer.write(example.SerializeToString())
            
print("\nDone")
print(true_max)
print(true_min)
print(phase_max)
print(phase_min)

Total: 289000
Done
0
100000
-100
100
